In [1]:
import pandas as pd
import numpy as np

## Reading and processing data from INPE Researcher And Publications

In [2]:
dataset_dir = '/mnt/d/Dados/OneDrive/IBICT/datasets/inpe/'
file_researchers = 'Lattes_Metadata.csv'
file_pubs = 'Lattes_Papers.csv'

In [3]:
encoding = "ISO-8859-1"

sel_cols=['i16','year','title','journal']

def converter(val):
    if val == np.nan:
        return 0
    return val

dfr = pd.read_csv(dataset_dir+file_researchers,sep=',',encoding=encoding)
dfp = pd.read_csv(dataset_dir+file_pubs,sep=',',
                    usecols=sel_cols,
                    # dtype={"year": "int16"},
                    # chunksize=1000,
                    encoding=encoding)

/tmp/ipykernel_354/2107384625.py:11: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfp = pd.read_csv(dataset_dir+file_pubs,sep=',',


In [4]:
researchers = dfr.set_index(['i16'])
researchers.sort_index(inplace=True)

## Populate Dπ-dev net

In [5]:
import os
import time
import logging

from tqdm.notebook import tqdm

from web3.providers.eth_tester import EthereumTesterProvider
from web3 import Web3
from web3.auto import w3

from web3.exceptions import TransactionNotFound
from eth_tester import PyEVMBackend
from hexbytes import HexBytes


from libs import compile_contract
from eth_tester.exceptions import TransactionFailed

### Methods

In [6]:
def get_tx_params(gas,account,chain_id=1337,min_gas_price='100'):
    # nonce = w3.eth.getTransactionCount(acount.address)
    nonce = w3.eth.getTransactionCount(w3.toChecksumAddress(account.address))
    tx_params = {'from': account.address,
                # 'to': contractAddress,
                'nonce': nonce,
                'gas': gas * 2,
                'gasPrice': w3.toWei(min_gas_price, 'gwei'), # defaul min gas price to besu (nao funciona w3.eth.gas_price)
                'chainId': chain_id
    }
    return tx_params

### Execution

Network configuration

In [7]:
# virtual env
# w3 = Web3(EthereumTesterProvider(PyEVMBackend()))

# Test Env
# HTTPProvider:
from web3 import Web3, IPCProvider
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

chain_id=1337

account importing

In [8]:
privateKey='c87509a1c067bbde78beb793e6fa76530b6382a4c0241e5e4a9ec0a0f44dc0d3'
acount = w3.eth.account.privateKeyToAccount(privateKey)
# w3.eth.get_balance(acount.address)

Contract setup

In [9]:
researcher_db_addr = '0xC8c03647d39a96f02f6Ce8999bc22493C290e734'
publication_db_addr = '0x446e083592D2dFa6661E20097F6898990C3CACE5'
# researcher_db_addr = '0xc0ED63E3A70BfCB003452B1Cc083db822e1f23e1'

In [10]:
dapp_dir = '..'+os.sep+'Dapp'+os.sep+'contracts'+os.sep

dapp_libs = [ dapp_dir + 'libs' + os.sep + 'EntitiesLib.sol' , dapp_dir + 'util' +os.sep+ 'HitchensUnorderedKeySet.sol']

contract_interface = compile_contract('ResearcherDB.sol',dapp_dir,dapp_libs)
print('ResearcherDB :: ', researcher_db_addr)
researcher_db_contract = w3.eth.contract(address=researcher_db_addr, abi=contract_interface["abi"])

# PUblications
contract_interface = compile_contract('PublicationDB.sol',dapp_dir,dapp_libs)
print('PublicationDB :: ', publication_db_addr)
publication_db_contract = w3.eth.contract(address=publication_db_addr, abi=contract_interface["abi"])


ResearcherDB ::  0xC8c03647d39a96f02f6Ce8999bc22493C290e734
PublicationDB ::  0x446e083592D2dFa6661E20097F6898990C3CACE5


populate

In [14]:
publications = dfp

tempo_pesquisador = []
tempo_publicacao = []

for titulo in tqdm(publications.title.unique()[4000:]):
    ldf = publications[publications.title == titulo]
    ano = ldf.year.head(1).values[0]
    
    authors = []
    #recuperando e criando os autores
    for row in ldf.itertuples():
        try:
            researcher_name = researchers.loc[row.i16]['name']
            start = time.time()
            
            # try:
            exist_flag, author_id = researcher_db_contract.functions.getResearcherByName(researcher_name).call()
            # except ValueError:
            # except:
            #     est_gas = researcher_db_contract.functions.createResearcher(researcher_name).estimateGas()
            #     tx_params = get_tx_params(est_gas,acount,chain_id=chain_id)
            #     tx = researcher_db_contract.functions.createResearcher(researcher_name).buildTransaction(tx_params)
            #     signed_tx = acount.signTransaction(tx)
            #     tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
            #     receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

            if exist_flag == False:
                #estimate de gas
                est_gas = researcher_db_contract.functions.createResearcher(researcher_name).estimateGas()
                #build the tx params
                tx_params = get_tx_params(est_gas,acount,chain_id=chain_id)

                tx = researcher_db_contract.functions.createResearcher(researcher_name).buildTransaction(tx_params)
                signed_tx = acount.signTransaction(tx)
                # send the transaction
                tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
                receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
                
                
            end_create_researcher = time.time() - start
            
            debug_line = [Web3.toHex(author_id),est_gas,end_create_researcher]
            tempo_pesquisador.append(debug_line)
            authors.append(author_id)
        
            try:
                start = time.time()
                est_gas = publication_db_contract.functions.createPublication(titulo,int(ano),'journal',authors).estimateGas()
                tx_params = get_tx_params(est_gas,acount,chain_id=chain_id)
                tx = publication_db_contract.functions.createPublication(titulo,int(ano),'journal',authors).buildTransaction(tx_params)
                
                signed_tx = acount.signTransaction(tx)
                tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
                receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

                end_create_pub = time.time() - start
                debug_line = [receipt['transactionHash'],est_gas,end_create_researcher]
                tempo_publicacao.append(end_create_pub)
                time.sleep(.2)
                
            except TransactionFailed:
            # except ValueError:
                print('    JA EXISTE')
                print('      ' + titulo)
                # print(row)
            # row.journal
        except ValueError as e: 
            # {'code': -32005, 'message': 'Transaction gas limit exceeds block gas limit'}
            print(e)
    


#
tempo_pesquisador = pd.DataFrame(tempo_pesquisador)
tempo_pesquisador.columns = ['time']
tempo_pesquisador.to_csv('tempo_pesquisador.csv')

tempo_publicacao = pd.DataFrame(tempo_publicacao)
tempo_publicacao.columns = ['time']
tempo_publicacao.to_csv('tempo_publicacao.csv')

  0%|          | 0/297272 [00:00<?, ?it/s]

TimeExhausted: Transaction HexBytes('0x4b3717e6eeb13c169427b201295ccf8d12e59192c88053acbd26d22e974d51d9') is not in the chain after 120 seconds

### Other contract Iteractions

In [ ]:
# transaction={'from': acount.address, 'gas': 100000, 'to': researcher_db_contract.address}
# estimated_gas = cc.functions.compareEntities(int(1), int(3)).estimateGas()
# researcher_db_contract.functions.getResearcherByName(researcher_name).call(transaction)
researcher_db_contract.functions.getResearcherByName('Ze das couves').call()

In [ ]:
#estimate de gas
est_gas = researcher_db_contract.functions.createResearcher('Maria').estimateGas()
            #build the tx params
tx_params = get_tx_params(est_gas,acount,chain_id=chain_id)

tx = researcher_db_contract.functions.createResearcher('Maria').buildTransaction(tx_params)
signed_tx = acount.signTransaction(tx)
# send the transaction
tx_hash = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
receipt

In [ ]:
researcher_db_contract.functions.getResearcherByName('Maria').call()


In [ ]:
id = researcher_db_contract.functions.getResearcherIdAtIndex(0).call()
researcher_db_contract.functions.getResearcherById(id).call()

In [ ]:
researcher_db_contract.functions.countResearchers().call()

In [15]:
publication_db_contract.functions.countPublications().call()

3830

In [ ]:
idp1 = publication_db_contract.functions.getPublicationIdAtIndex(1).call()
publication_db_contract.functions.getPublicationById(idp1).call()

In [ ]:
id = b'\x18\xa1\x8a\xac\x0e\x11\xeb\x92\xa7\xb0e\xf81\x04\x90G\xf5\x1f\xb0\xd0\x88yyG\x81+#\xff\xa6\\g\xd2'
Web3.toHex(id)

In [ ]:
researcher_db_contract.functions.getResearcherById(id).call()